In [2]:
REPOSITORY_PATH = 'BoostingCNN-Jpeg-Primary-Quantization-Matrix-Estimation'

In [3]:
import os
import cv2
import numpy as np
import pandas as pd
from glob import glob

from core.utility import preprocess_input, estimation_by_mse, image_by_qf_generator, read_and_preprocess_image, double_compress_image, compress_image
from core.model import Model

map_90 - [(60, 90), (98, 90)]

map_80 - [(50, 80), (98, 80)]

In [4]:
modelname = 'model_QF1_60-98_QF2_90-2-term-loss.h5'
qf_map_name = 'qf1_qf2_map_90.npy'
model_path = os.path.join(REPOSITORY_PATH, 'models', modelname)
qf_map_path = os.path.join(REPOSITORY_PATH, 'resources', qf_map_name)

In [5]:
model = Model(model_path, qf_map_path)

In [6]:
img_path = 'BoostingCNN-Jpeg-Primary-Quantization-Matrix-Estimation/resources/00000000_redaf7d93t.TIF_85_90.png'
image = read_and_preprocess_image(img_path, target_size=(64, 64))

### Написать функцию оценивания 𝑄𝐹1 по вектору из 15 оценок 

Для оценки QF_1 используется следующая функция:
```python
def estimation_by_mse(x, y, weights=None, used_coeffs=slice(16)):
    """
    Estimates QF for given vector of DCT coeffs and mapping of known DCT coeffs to QF estimations

    Args:
        x (numpy.array): vector of DCT coeffs to estimate
        y (numpy.array): QF - DCT mapping
        weights (numpy.array, optional): Weights used for weighted MSE computation. Defaults to None.
        used_coeffs (slice or List[int], optional): Positions of coeffs to be used. Defaults to slice(16).

    Returns:
        numpy.array: (W)MSE values calculated for given DCT coeffs and DCT-QF mapping
    """
    diff = np.asarray((x - y) ** 2)
    return np.average(diff[::, used_coeffs], axis=1, weights=weights[used_coeffs] if weights is not None else weights)
```

In [7]:
model.estimate_qf1(image, lambda x, y: estimation_by_mse(x, y, weights=None, used_coeffs=slice(16)))

1/1 [==============================] - 0s 496ms/step


array([79, 90], dtype=object)

In [8]:
target_size = (64, 64)
image_fname = 'data/1.TIF'
in_image = cv2.imread(image_fname)
image = preprocess_input(double_compress_image(in_image, 80, 90)[:64, :64], target_size)
model.estimate_qf1(image, lambda x, y: estimation_by_mse(x, y, weights=None, used_coeffs=slice(16))), model.predict_coeffs(image)

1/1 [==============================] - 0s 42ms/step


(array([77, 90], dtype=object),
 array([ 7,  5,  6,  6,  6,  5,  7,  6,  6,  6,  8,  8,  7,  9, 11]))

### Программно сгенерировать 5 принципиально различных ситуаций

In [9]:
test_cases = [
    (70, ),
    (85, 90),
    (60, 90),
    (95, 90),
    (89, 90)
]

test_images = []
for test_case in test_cases:
    for filename in glob('data/*.tif'):
        image = cv2.imread(filename)
        if len(test_case) == 1:
            test_images.append([test_case, filename, compress_image(image, *test_case)])
        else:
            test_images.append([test_case, filename, double_compress_image(image, *test_case)])

In [10]:
len(test_images)

50

### Сравнить ошибки оценивания 𝑄𝐹1 в различных ситуация

In [17]:
target_size = (64, 64)

experiment_results = []
for idx, (initial_qf, filename, image) in enumerate(test_images):
    image = preprocess_input(image[:target_size[0], :target_size[1]], target_size)
    preds = model.estimate_qf1(image, lambda x, y: estimation_by_mse(x, y, weights=None, used_coeffs=slice(16)))
    error = np.abs(initial_qf[0] - preds[0])
    experiment_results.append((filename, initial_qf, preds, error))

1/1 [==============================] - 0s 36ms/step


In [ ]:
experiment_results

In [18]:
df = pd.DataFrame(experiment_results, columns=['filename', 'initial_qf', 'predicted_qf', 'error'])
df.head()

,filename,initial_qf,predicted_qf,error
0,data/0.tif,"(70,)","[67, 90]",3
1,data/1.tif,"(70,)","[70, 90]",0
2,data/2.tif,"(70,)","[70, 90]",0
3,data/3.tif,"(70,)","[68, 90]",2
4,data/4.tif,"(70,)","[74, 90]",4


In [19]:
df

,filename,initial_qf,predicted_qf,error
0,data/0.tif,"(70,)","[67, 90]",3
1,data/1.tif,"(70,)","[70, 90]",0
2,data/2.tif,"(70,)","[70, 90]",0
3,data/3.tif,"(70,)","[68, 90]",2
4,data/4.tif,"(70,)","[74, 90]",4
5,data/5.tif,"(70,)","[73, 90]",3
6,data/6.tif,"(70,)","[73, 90]",3
7,data/7.tif,"(70,)","[73, 90]",3
8,data/8.tif,"(70,)","[66, 90]",4
9,data/9.tif,"(70,)","[70, 90]",0


In [22]:
print('Mean error across all cases: \t', df['error'].mean())

Mean error across all cases: 	 2.74


In [29]:
print('Mean error for each case: \t')
df.groupby('initial_qf').mean()

Mean error for each case: 	


<ipython-input-29-02b6f199d939>:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby('initial_qf').mean()


,error
initial_qf,
"(60, 90)",3.1
"(70,)",2.2
"(85, 90)",1.4
"(89, 90)",5.7
"(95, 90)",1.3
